In [ ]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack_integrations.components.generators.ollama import OllamaGenerator
from pathlib import Path

# Meta Parameters

In [ ]:
if not 'process_text' in locals():
    process_text = ""

if not 'process_name' in locals():
    process_name = ""

if not 'large_language_model' in locals():
    large_language_model = ""

if not 'prompt_name' in locals():
    prompt_name = ""

if not 'loop_limit' in locals():
    loop_limit = "20"

if not 'llm_url' in locals():
    llm_url = "http://localhost:11434"


# comp-gpu06:8765

# print("process_name:", process_name)
# print("process description: ", process_text)
# print("llm:" , large_language_model)
# print("prompt_name:", prompt_name)

# Roles

In [ ]:
#Context
prompt_Role = """
[Context]
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response).
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Role = prompt_Role + """
{{ task }}
"""
task = """
Your task is to find all roles or actors for a DCR graph in the process description I will provide you with.
"""

# Output / Format
prompt_Role = prompt_Role + """
Return the roles in an array in JSON format, similar to: {"roles": [...,...,...]}. Do not forget the delimiter "," between the identified roles. Do not use any other delimiter. Do not add any other information to your answer. Do not include duplicate roles. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Role = prompt_Role + """
The process description in question is:
{{ process }}
"""

## Combine Role Pipeline

In [ ]:
prompt_builder = PromptBuilder(template=prompt_Role)
llm = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

role_pipeline = Pipeline()
role_pipeline.add_component("prompt_builder", prompt_builder)
role_pipeline.add_component("llm", llm)
role_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

# Prompt for Activities

In [ ]:
#Context
prompt_Activity = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response).
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Activity = prompt_Activity + """
Your task is to find all activities or events for a DCR graph in the process description I will provide you with and map them to the following roles: {{ roles }}.
"""

# Output / Format
prompt_Activity = prompt_Activity + """
Return the activitites as keys in key-value-pairs in JSON format, with their associated role as value, which should look like this: { {"activity": ..., "role": ...}, ...}. Do not forget the delimiter "," between the identified activities. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate activities. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Activity = prompt_Activity + """
The process description in question is:
{{ process }}
"""

## Combine Activity Pipeline

In [ ]:
prompt_builder_Activity = PromptBuilder(template=prompt_Activity)
llm2 = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

activity_pipeline = Pipeline()
activity_pipeline.add_component("prompt_builder_Activity", prompt_builder_Activity)
activity_pipeline.add_component("llm2", llm2)
activity_pipeline.connect("prompt_builder_Activity", "llm2")

🚅 Components
  - prompt_builder_Activity: PromptBuilder
  - llm2: OllamaGenerator
🛤️ Connections
  - prompt_builder_Activity.prompt -> llm2.prompt (str)

# Prompt for Rules

In [ ]:
#Context
prompt_Rules = """
Imaging you are an expert in Business Process Management with extensive knowledge about declarative process modelling. Additionally, you have specific knowledge about the modelling technique "DCR" (Dynamic Condition Response).
I am going to feed you a textual description of some sort of Business Process. It could be a process in a company or organization. 
"""

#Task
prompt_Rules = prompt_Rules + """
Your task is to find and define all constraints between the activities for a DCR graph in the process description I will provide you with, based on the already identified activities: {{ activities }}.
A constraint must be in accordance with DCR a Condition, Response, Exclude, Include or Milestone, and its associated activities are a source activity and a target activity.
"""

# Output / Format
prompt_Rules = prompt_Rules + """
Return the DCR constraints as keys in key-value-pairs in JSON format, which should look like this: { {"constraint": ..., "source activity: ..., "target activity": ...}, ...}. The constraint should be one of the constraint types of DCR and the activities its associated source and target activity. Do not forget the delimiter "," between the identified constraints. Do not use any other delimiter. Do not alter the text. Do not add any other information to your answer. Do not include duplicate constraints. Do not include the roles of the activities in the output. Do not repeat the task description in your answer. Do neither describe nor explain your answer. 
"""

# Process Description
prompt_Rules = prompt_Rules + """
The process description in question is:
{{ process }}
"""

## Combine Rule Pipeline

In [ ]:
from haystack import Pipeline
from haystack import component

prompt_builder_Rules = PromptBuilder(template=prompt_Rules)
llm3 = OllamaGenerator(model=large_language_model, url=llm_url, timeout=1200)

rule_pipeline = Pipeline()
rule_pipeline.add_component("prompt_builder_Rules", prompt_builder_Rules)
rule_pipeline.add_component("llm3", llm3)
rule_pipeline.connect("prompt_builder_Rules", "llm3")

🚅 Components
  - prompt_builder_Rules: PromptBuilder
  - llm3: OllamaGenerator
🛤️ Connections
  - prompt_builder_Rules.prompt -> llm3.prompt (str)

# Graph Generation

In [ ]:
import json
import xml.etree.ElementTree as ET 
import uuid
from pathlib import Path
from datetime import datetime

## Run Prompt Pipeline

In [ ]:
def runPromptPipeline(): 
    start_time_role = datetime.now()
    response_Roles = role_pipeline.run(
        {
            "prompt_builder": {"task": task, "process": process_text},
        }
    )
    processing_time_role = datetime.now() - start_time_role
    processing_duration_role = processing_time_role.total_seconds()
    input_tokens_Role = response_Roles["llm"]["meta"][0]["prompt_eval_count"]
    output_tokens_Role = response_Roles["llm"]["meta"][0]["eval_count"]
    cleaned_roles = ''.join(response_Roles["llm"]["replies"])
    cleaned_roles = cleaned_roles.replace("'", '').replace("{", '').replace("}", '').replace("[", '').replace("]", '').replace(":", '').replace("\"roles\"", '').replace("\n",'').replace(" ", '').replace("\",\"",'\", \"').replace("\\",'')

    start_time_activity = datetime.now()
    response_Activities = activity_pipeline.run(
        {
            "prompt_builder_Activity": {"process": process_text, "roles": cleaned_roles},
        }
    )
    processing_time_activity = datetime.now() - start_time_activity
    processing_duration_activity = processing_time_activity.total_seconds()
    input_tokens_Activity = response_Activities["llm2"]["meta"][0]["prompt_eval_count"]
    output_tokens_Activity = response_Activities["llm2"]["meta"][0]["eval_count"]
    cleaned_activities = ''.join(response_Activities["llm2"]["replies"])
    cleaned_activities = cleaned_activities.replace("'", '').replace("[", '').replace("]", '').replace("\"roles\"", '').replace("\n",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"activity','},{"activity').replace('} {"activity','},{"activity')

    start_time_rule = datetime.now()
    response_Rules = rule_pipeline.run(
        {
            "prompt_builder_Rules": {"process": process_text, "activities": cleaned_activities},#{"question": question},
        }
    )
    processing_time_rule = datetime.now() - start_time_rule
    processing_duration_rule = processing_time_rule.total_seconds()
    input_tokens_Rule = response_Rules["llm3"]["meta"][0]["prompt_eval_count"]
    output_tokens_Rule = response_Rules["llm3"]["meta"][0]["eval_count"]
    cleaned_rules = ''.join(response_Rules["llm3"]["replies"])
    cleaned_rules = cleaned_rules.replace("'", '').replace("[", '').replace("]", '').replace("\n",'').replace("  ",'').replace("\",\"",'\", \"').replace("\\",'').replace('}{"constraint','},{"constraint').replace('} {"constraint','},{"constraint')

    return cleaned_roles, processing_duration_role, input_tokens_Role, output_tokens_Role, cleaned_activities, processing_duration_activity, input_tokens_Activity, output_tokens_Activity, cleaned_rules, processing_duration_rule, input_tokens_Rule, output_tokens_Rule


## Create JSON from Output

In [ ]:
def generateJSON(roles,activities,rules):
    rolesString = '"roles": [' + roles + ']'
    activitiesString = '"activities": [' + activities + ']'
    rulesString = '"constraints": [' + rules + ']'
    graphString = '{"DCRgraph": {' + rolesString + ', ' + activitiesString + ', ' + rulesString + '} }'
    print(graphString, flush=True)
    graphJSON = json.loads(graphString)
    
    # print(rolesString)
    # print(activitiesString)
    # print(rulesString)
    # print(graphString)

    return graphJSON


## Generate DCR Editor XML

In [ ]:
### the code for json_to_xml() was mostly generated using GPT-4o mini
def json_to_xml(json_obj):
    # Create the root element
    definitions = ET.Element("dcr:definitions", {
        "xmlns:dcr": "http://tk/schema/dcr",
        "xmlns:dcrDi": "http://tk/schema/dcrDi",
        "xmlns:dc": "http://www.omg.org/spec/DD/20100524/DC"
    })

    # Create dcrGraph element
    dcr_graph = ET.SubElement(definitions, "dcr:dcrGraph", id="dcrGraph")

    # Create events
    event_ids = {}
    for activity in json_obj["DCRgraph"]["activities"]:
        role = activity["role"]
        description = activity["activity"]
        event_id = f"Event_{uuid.uuid4().hex[:8]}"  # Generate a unique event ID
        event_ids[event_id] = activity  # Store the event ID with its activity for later use

        event = ET.SubElement(dcr_graph, "dcr:event", {
            "id": event_id,
            "role": role,
            "description": description,
            "included": "true",
            "executed": "false",
            "pending": "false",
            "enabled": "false"
        })

    # Create relations
    relation_ids = []
    for constraint in json_obj["DCRgraph"]["constraints"]:
        source_activity = constraint["source activity"]
        target_activity = constraint["target activity"]
        relation_id = f"Relation_{uuid.uuid4().hex[:8]}"  # Generate a unique relation ID

        #check whether constraint type matches DCR Editor XML types
        if constraint["constraint"] == "Condition":
            constraint_type = "condition"
        elif constraint["constraint"] == "Response":
            constraint_type = "response"
        elif constraint["constraint"] == "Exclude":
            constraint_type = "exclude"
        elif constraint["constraint"] == "Include":
            constraint_type = "include"
        elif constraint["constraint"] == "Milestone":
            constraint_type = "milestone"
        else:
            raise TypeError("Constraint Type was not identified correctly")
        
        # Find the corresponding event IDs
        source_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == source_activity), None)
        target_event_id = next((eid for eid, act in event_ids.items() if act["activity"] == target_activity), None)

        if source_event_id and target_event_id:
            # Create the dcr:relation with a unique ID
            ET.SubElement(dcr_graph, "dcr:relation", {
                "id": relation_id,
                "type": constraint_type, 
                "sourceRef": source_event_id,
                "targetRef": target_event_id
            })
            relation_ids.append(relation_id)  # Store the relation ID for later use

    # Create dcrRootBoard element
    root_board = ET.SubElement(definitions, "dcrDi:dcrRootBoard", id="RootBoard")
    plane = ET.SubElement(root_board, "dcrDi:dcrPlane", id="Plane", boardElement="dcrGraph")

    # Create shapes for the diagram
    for event_id in event_ids.keys():
        shape = ET.SubElement(plane, "dcrDi:dcrShape", id=f"{event_id}_di", boardElement=event_id)
        # Example bounds, you can adjust the coordinates as needed
        ET.SubElement(shape, "dc:Bounds", x="640", y="230", width="130", height="150")

    # Create dcrDi:relation elements with unique IDs
    for relation_id in relation_ids:
        # Create a corresponding dcrDi:relation
        relation_di_id = f"{relation_id}_di"  # ID for dcrDi:relation
        relation_di = ET.SubElement(plane, "dcrDi:relation", id=relation_di_id, boardElement=relation_id)
        # Example waypoints, you can adjust the coordinates as needed
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="380")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="705", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="400")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="620", y="305")
        ET.SubElement(relation_di, "dcrDi:waypoint", x="640", y="305")

    # Convert to string
    xml_str = ET.tostring(definitions, encoding='UTF-8', xml_declaration=True).decode('UTF-8')
    return xml_str



## Combined Graph Generation

In [ ]:
current_run = 0
generation_start_time = datetime.now()
errors = 0
role_times = []
role_input_token = []
role_output_token = []
activity_times = []
activity_input_token = []
activity_output_token = []
rule_times = []
rule_input_token = []
rule_output_token = []

while True:
    if current_run >= loop_limit:
        # safety mechanism to ensure termination if an unexpected issue is not handled
        # important to leave in when running everything automatically
        raise ValueError("Exceeded maximumg number of runs. Generation of Graph failed.")
    try:
        roles, role_time, role_input, role_output, activities, activity_time, activity_input, activity_output, rules, rule_time, rule_input, rule_output = runPromptPipeline()
        role_times.append(role_time)
        role_input_token.append(role_input)
        role_output_token.append(role_output)
        activity_times.append(activity_time)
        activity_input_token.append(activity_input)
        activity_output_token.append(activity_output)
        rule_times.append(rule_time)
        rule_input_token.append(rule_input)
        rule_output_token.append(rule_output)
        graphJSON = generateJSON(roles,activities,rules)
        print("Parsing to JSON successful")
        xml = json_to_xml(graphJSON)
        print(f"XML Generation successful ({process_name},Pipeline_A1_{prompt_name})")
        print("xml:", xml)
        print("\n")
        break
    except Exception as e:
        current_run += 1
        errors += 1
        print(f"Graph Generation failed with error ({process_name},Pipeline_A1_{prompt_name})")
        print(e, flush=True)
        runs_left = loop_limit - current_run
        print("Iterations left: " + str(runs_left))
        print("\n")

generation_finished_time = datetime.now() - generation_start_time
generation_duration = generation_finished_time.total_seconds()

## Save Time to Results

In [ ]:
new_times = f"{process_name};{generation_duration};{role_times};{role_input_token};{role_output_token};{activity_times};{activity_input_token};{activity_output_token};{rule_times};{rule_input_token};{rule_output_token};{errors}\n"

folder = Path('./6_Time_Analysis') / "data" / large_language_model / "Pipeline_A1"
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{prompt_name}.txt"
file_path = folder / file_name 

with open(file_path, 'a+') as file:
    file.write(new_times)

## Save XML to Results

In [ ]:
folder = Path(result_xml_path) / large_language_model / "Pipeline_A1" / prompt_name
folder.mkdir(parents=True, exist_ok=True) 

file_name = f"{process_name}_{large_language_model}_{prompt_name}_{datetime.now().date()}_{datetime.now().time()}.xml"
file_path = folder / file_name 

with open(file_path, 'w') as result_destination:
    result_destination.write(xml)
